<a href="https://colab.research.google.com/github/VineetRoy1106/PDF-Q-A-Hybrid-Search/blob/main/RAG_Pinecone_Hybrid_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --upgrade --quiet pinecone-client pinecone-text pinecone-notebooks

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.0 MB/s eta 0:00:00


In [11]:
# setting Pinecone API Key
from google.colab import userdata
api_key = userdata.get('PINECONE_API_KEY')

In [7]:
!pip install langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully u

In [8]:
# Pinecone Hybrid Search
from langchain_community.retrievers import PineconeHybridSearchRetriever


### Setting up Pinecone Index

In [12]:
import os
from pinecone import Pinecone,ServerlessSpec

index_name = "langchain-pinecone-hybrid-search"

# initialize Pinecone client
pc = Pinecone(api_key=api_key)

# create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # dimensionality of dense model
        metric="dotproduct",  # sparse values supported only for dotproduct
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

In [13]:
index = pc.Index(index_name)


In [14]:
index

In [17]:
pip install langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 7.8 MB/s eta 0:00:00


### Hugging Face Embedding

In [19]:
# vector embedding and sparse matrix

from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [21]:
#sparse matrix
from pinecone_text.sparse import BM25Encoder


bm25_encoder = BM25Encoder().default()

In [22]:
!pip install -qU langchain_community wikipedia

  Preparing metadata (setup.py) ... done


In [23]:
from langchain_community.document_loaders import WikipediaLoader

Using Wikipedia loader for loading Dataset

In [24]:
docs = WikipediaLoader(query="Lionel Messi", load_max_docs=2).load()
len(docs)

2

In [26]:
docs[0].metadata

{'title': 'Lionel Messi',
 'summary': 'Lionel Andrés "Leo" Messi (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ; born 24 June 1987) is an Argentine professional footballer who plays as a forward for and captains both Major League Soccer club Inter Miami and the Argentina national team. Widely regarded as one of the greatest players of all time, Messi set numerous records for individual accolades won throughout his professional footballing career such as eight Ballon d\'Or awards and eight times being named the world\'s best player by FIFA. He is the most decorated player in the history of professional football having won 45 team trophies, including twelve league titles, four UEFA Champions Leagues, two Copa Américas, and one FIFA World Cup. Messi holds the records for most European Golden Shoes (6), most goals for a single club (672, with Barcelona), most goals (474), hat-tricks (36) and assists (192) in La Liga, most matches played (39), assists (18) and goal contributions (34) in t

In [37]:
docs[0].page_content  # a part of the page content

'Lionel Andrés "Leo" Messi (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ; born 24 June 1987) is an Argentine professional footballer who plays as a forward for and captains both Major League Soccer club Inter Miami and the Argentina national team. Widely regarded as one of the greatest players of all time, Messi set numerous records for individual accolades won throughout his professional footballing career such as eight Ballon d\'Or awards and eight times being named the world\'s best player by FIFA. He is the most decorated player in the history of professional football having won 45 team trophies, including twelve league titles, four UEFA Champions Leagues, two Copa Américas, and one FIFA World Cup. Messi holds the records for most European Golden Shoes (6), most goals for a single club (672, with Barcelona), most goals (474), hat-tricks (36) and assists (192) in La Liga, most matches played (39), assists (18) and goal contributions (34) in the Copa América, most matches played (

In [38]:
dataset = [docs[0].page_content , docs[1].page_content]

In [39]:


# fit tf-idf values on the Dataset
bm25_encoder.fit(dataset)

# store the values to a json file
bm25_encoder.dump("bm25_values.json")

# load to your BM25Encoder object
bm25_encoder = BM25Encoder().load("bm25_values.json")

  0%|          | 0/2 [00:00<?, ?it/s]

### Load Retriever

In [40]:
retriever = PineconeHybridSearchRetriever(
    embeddings=embeddings, sparse_encoder=bm25_encoder, index=index
)

In [41]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x78f73dff3a60>, index=<pinecone.data.index.Index object at 0x78f855d939a0>)

In [42]:
retriever.add_texts(dataset)

  0%|          | 0/1 [00:00<?, ?it/s]

In [44]:
result = retriever.invoke("In which year messi made his debut")

In [46]:
result[0]

Document(metadata={'score': 0.558901906}, page_content='Lionel Andrés "Leo" Messi (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ; born 24 June 1987) is an Argentine professional footballer who plays as a forward for and captains both Major League Soccer club Inter Miami and the Argentina national team. Widely regarded as one of the greatest players of all time, Messi set numerous records for individual accolades won throughout his professional footballing career such as eight Ballon d\'Or awards and eight times being named the world\'s best player by FIFA. He is the most decorated player in the history of professional football having won 45 team trophies, including twelve league titles, four UEFA Champions Leagues, two Copa Américas, and one FIFA World Cup. Messi holds the records for most European Golden Shoes (6), most goals for a single club (672, with Barcelona), most goals (474), hat-tricks (36) and assists (192) in La Liga, most matches played (39), assists (18) and goal contri

In [48]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 4.2 MB/s eta 0:00:00


### **Setting up llm for Fine-Tuning the result into precise Answer**

In [58]:
from langchain_groq import ChatGroq

from google.colab import userdata
groq_api_key = userdata.get('GROQ_API_KEY')
llm=ChatGroq(groq_api_key=groq_api_key,model_name="Llama3-8b-8192")

In [59]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x78f75527c8e0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x78f75527d480>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [60]:
# from langchain.chains.question_answering import load_qa_chain
# chain = load_qa_chain(llm, chain_type="stuff")

# def get_answer(query):
#   similar_docs = retriever.invoke(query)
#   # print(similar_docs)
#   answer =  chain.run(input_documents=docs, question=query)
#   return  answer

# query = "In which year messi made his debut"
# get_answer(query)

<ipython-input-60-5ab894e1eed4>:2: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain = load_qa_chain(llm, chain_type="stuff")
<ipython-input-60-5ab894e1eed4>:7: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer =  chain.run(input_documents=docs, question=query)


'According to the provided text, Lionel Messi made his competitive debut at age 17 in October 2004.'

In [62]:
# from langchain.chains.question_answering import load_qa_chain

# # Load the QA chain with the new chain_type (use 'stuff' or others as needed)
# chain = load_qa_chain(llm, chain_type="stuff")

# def get_answer(query):
#     # Step 1: Retrieve relevant documents
#     similar_docs = retriever.invoke(query)

#     # Step 2: Pass the documents and the query to the updated chain
#     answer = chain({"input_documents": similar_docs, "question": query})

#     return answer['output_text']

# query = "In which year did Messi make his debut?"
# get_answer(query)


<ipython-input-62-9c333622add8>:11: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = chain({"input_documents": similar_docs, "question": query})


'According to the text, Lionel Messi made his competitive debut at age 17 in October 2004.'

###**Loading the Q/A Chain**

In [63]:
from langchain.chains.question_answering import load_qa_chain

# Load the QA chain with the correct chain type (e.g., 'stuff')
chain = load_qa_chain(llm, chain_type="stuff")

def get_answer(query):
    # Step 1: Retrieve relevant documents
    similar_docs = retriever.invoke(query)

    # Step 2: Pass the documents and the query to the updated chain
    answer = chain.invoke({"input_documents": similar_docs, "question": query})

    return answer['output_text']

query = "In which year did Messi make his debut?"
get_answer(query)


'According to the text, Messi made his competitive debut at age 17 in October 2004.'

In [64]:
query = "how many ballondor does messi have"
get_answer(query)

"According to the provided context, Lionel Messi has won 8 Ballon d'Or awards."

In [65]:
query = "After Barcelona, which club does messi played for"
get_answer(query)

'After Barcelona, Lionel Messi played for Paris Saint-Germain (August 2021) and then for Inter Miami (July 2023).'

In [66]:
query = "what is the full name of messi ?"
get_answer(query)

'The full name of Lionel Messi is Lionel Andrés Messi.'

In [67]:
query = "Birthplace of messi of messi ?"
get_answer(query)

'According to the provided context, Lionel Messi was born in Rosario, Argentina.'